Anova_tfidf_250features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:

#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/tfidf_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,1,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,2,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,3,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,4,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head() 

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,-0.052757,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,-0.052757,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,-0.052757,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,-0.052757,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,-0.086085,-0.106214,-0.086085,-0.20195,-0.040048,-0.086085,-0.08741,-0.127371,-0.104537,-0.052757,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3815162863449745


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.345082,0.345082,0.758992,0.234606,0.29771,0.29771,0.323669,0.172632


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9886    0.9640    0.9762       361
           2     0.9596    0.9873    0.9733       553
           3     0.9833    0.9608    0.9719       306

    accuracy                         0.9738      1220
   macro avg     0.9772    0.9707    0.9738      1220
weighted avg     0.9741    0.9738    0.9738      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3730    0.2994    0.3322       157
           2     0.4524    0.6414    0.5305       237
           3     0.2581    0.1231    0.1667       130

    accuracy                         0.4103       524
   macro avg     0.3612    0.3546    0.3431       524
weighted avg     0.3804    0.4103    0.3808       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.345082,0.345082,0.758992,0.234606,0.297710,0.297710,0.323669,0.172632
1,RandomForest,0.973770,0.973770,0.974121,0.973777,0.410305,0.410305,0.380393,0.380827


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5037    0.1884    0.2742       361
           2     0.4854    0.8409    0.6155       553
           3     0.5591    0.2320    0.3279       306

    accuracy                         0.4951      1220
   macro avg     0.5160    0.4204    0.4059      1220
weighted avg     0.5093    0.4951    0.4424      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2568    0.1210    0.1645       157
           2     0.4275    0.7089    0.5333       237
           3     0.3333    0.1462    0.2032       130

    accuracy                         0.3931       524
   macro avg     0.3392    0.3253    0.3003       524
weighted avg     0.3530    0.3931    0.3409       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.645875,0.323741,0.456107,0.456107,0.384889,0.305285
1,RandomForest,0.973770,0.973770,0.974430,0.973786,0.402672,0.402672,0.375719,0.377770
2,Adaboost,0.495082,0.495082,0.509284,0.442376,0.393130,0.393130,0.352972,0.340924


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9245    0.4072    0.5654       361
           2     0.5734    0.9964    0.7279       553
           3     0.9500    0.3105    0.4680       306

    accuracy                         0.6500      1220
   macro avg     0.8160    0.5713    0.5871      1220
weighted avg     0.7717    0.6500    0.6146      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3288    0.1529    0.2087       157
           2     0.4499    0.7764    0.5697       237
           3     0.2619    0.0846    0.1279       130

    accuracy                         0.4179       524
   macro avg     0.3468    0.3380    0.3021       524
weighted avg     0.3670    0.4179    0.3519       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.645875,0.323741,0.456107,0.456107,0.384889,0.305285
1,RandomForest,0.973770,0.973770,0.974430,0.973786,0.402672,0.402672,0.375719,0.377770
2,Adaboost,0.495082,0.495082,0.509284,0.442376,0.393130,0.393130,0.352972,0.340924
3,GBM,0.650000,0.650000,0.771740,0.614606,0.417939,0.417939,0.366956,0.351913


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:

y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4856    0.6066    0.5394       361
           2     0.7139    0.4467    0.5495       553
           3     0.4539    0.6275    0.5267       306

    accuracy                         0.5393      1220
   macro avg     0.5511    0.5603    0.5386      1220
weighted avg     0.5811    0.5393    0.5408      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2872    0.3567    0.3182       157
           2     0.4678    0.3376    0.3922       237
           3     0.2658    0.3231    0.2917       130

    accuracy                         0.3397       524
   macro avg     0.3403    0.3391    0.3340       524
weighted avg     0.3636    0.3397    0.3451       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.345082,0.345082,0.758992,0.234606,0.297710,0.297710,0.323669,0.172632
1,RandomForest,0.973770,0.973770,0.974121,0.973777,0.410305,0.410305,0.380393,0.380827
2,SVM,0.539344,0.539344,0.581117,0.540807,0.339695,0.339695,0.363590,0.345062


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4816    0.5069    0.4939       361
           2     0.5805    0.6980    0.6338       553
           3     0.4400    0.2516    0.3202       306

    accuracy                         0.5295      1220
   macro avg     0.5007    0.4855    0.4826      1220
weighted avg     0.5160    0.5295    0.5138      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3239    0.3631    0.3423       157
           2     0.4570    0.5612    0.5038       237
           3     0.3509    0.1538    0.2139       130

    accuracy                         0.4008       524
   macro avg     0.3773    0.3594    0.3533       524
weighted avg     0.3908    0.4008    0.3835       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.645875,0.323741,0.456107,0.456107,0.384889,0.305285
1,RandomForest,0.973770,0.973770,0.974430,0.973786,0.402672,0.402672,0.375719,0.377770
2,Adaboost,0.495082,0.495082,0.509284,0.442376,0.393130,0.393130,0.352972,0.340924
3,GBM,0.650000,0.650000,0.771740,0.614606,0.417939,0.417939,0.366956,0.351913
4,SVM,0.555738,0.555738,0.593575,0.515959,0.414122,0.414122,0.368381,0.363071
5,KNN,0.529508,0.529508,0.515967,0.513758,0.400763,0.400763,0.390802,0.383498


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6346    0.0914    0.1598       361
           2     0.7339    0.1447    0.2417       553
           3     0.2823    0.9771    0.4381       306

    accuracy                         0.3377      1220
   macro avg     0.5503    0.4044    0.2799      1220
weighted avg     0.5913    0.3377    0.2667      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.0701    0.1158       157
           2     0.5098    0.1097    0.1806       237
           3     0.2455    0.8308    0.3789       130

    accuracy                         0.2767       524
   macro avg     0.3629    0.3368    0.2251       524
weighted avg     0.3913    0.2767    0.2104       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.645875,0.323741,0.456107,0.456107,0.384889,0.305285
1,RandomForest,0.973770,0.973770,0.974430,0.973786,0.402672,0.402672,0.375719,0.377770
2,Adaboost,0.495082,0.495082,0.509284,0.442376,0.393130,0.393130,0.352972,0.340924
3,GBM,0.650000,0.650000,0.771740,0.614606,0.417939,0.417939,0.366956,0.351913
4,SVM,0.555738,0.555738,0.593575,0.515959,0.414122,0.414122,0.368381,0.363071
5,KNN,0.529508,0.529508,0.515967,0.513758,0.400763,0.400763,0.390802,0.383498
6,GNB,0.337705,0.337705,0.591282,0.266724,0.276718,0.276718,0.391347,0.210370
